In [1]:
import torch
import transformers
import torchvision.models as models

from torch.profiler import profile, record_function, ProfilerActivity
from thop import profile as thop_profile 

In [4]:
DEVICE = "cuda:1"
BS = 64

In [3]:
resnet = models.resnet50().eval().to(DEVICE).eval()
rgb_inputs = torch.randn((BS,3,224,224)).to(DEVICE)


In [4]:
bert = transformers.BertModel.from_pretrained('bert-base-uncased').to(DEVICE).eval()
txt_inputs = torch.randint(0, 10000, size=(BS, 512)).to(DEVICE)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[[[-7.0565e-01,  8.6936e-01,  3.1174e-02,  ...,  8.0385e-01,
            1.0879e-01,  5.0865e-01],
          [-2.4124e-01, -5.3116e-01, -1.9429e+00,  ..., -1.0596e+00,
            4.0990e-01,  1.4967e+00],
          [-5.6151e-01,  4.5833e-01, -1.0023e+00,  ..., -1.2647e+00,
           -2.3268e+00,  1.7362e+00],
          ...,
          [-3.6343e-02,  3.1525e+00,  8.6315e-01,  ...,  1.2143e+00,
           -1.9257e+00, -9.9649e-01],
          [ 3.2646e-01,  1.6715e+00, -1.7885e-02,  ...,  1.7449e-01,
           -9.8891e-01, -1.8296e+00],
          [-8.2463e-01,  4.4293e-01, -1.3876e-01,  ..., -8.4585e-01,
            4.2261e-01,  1.3401e+00]],

         [[ 2.1816e+00,  1.3265e-01,  6.4581e-01,  ...,  5.7476e-01,
           -1.9074e+00,  2.7646e-01],
          [-3.5510e-01,  4.8271e-01,  1.4966e+00,  ...,  8.6373e-01,
            8.9565e-01, -1.3286e+00],
          [ 1.1402e+00,  1.5040e+00,  6.3792e-01,  ...,  5.7103e-01,
           -5.6752e-01,  8.0591e-02],
          ...,
     

In [11]:
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], 
             on_trace_ready=torch.profiler.tensorboard_trace_handler('log/resnet50-bs512'),
             record_shapes=True, with_flops=True, profile_memory=True) as prof:
    with record_function("model_inference"):
        resnet(rgb_inputs)